# Protein Secondary Structure Prediction with Recurrent Neural Networks

## 8 Class secondary structure prediction

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
from torch.utils import data
import torch.utils.data as utils
from torch.nn.utils.rnn import pad_sequence ,pack_padded_sequence,pad_packed_sequence
import random
import os
import math
import pickle
import time
import tqdm
from sklearn.metrics import accuracy_score,classification_report
from matplotlib import pyplot as plt
%matplotlib inline

### Set Random Number Initializers for Repeatability

In [ ]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
target_names=['L', 'B', 'E', 'G', 'I', 'H', 'S', 'T']
residue_names = ['A', 'C', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'M', 'L', 'N', 'Q', 'P', 'S', 'R', 'T', 'W', 'V', 'Y', 'X','NoSeq']

#### Set device for the model to run on.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)
cpu = torch.device("cpu")

### Dataloader


In [ ]:
def load_from_pkl(path):
    with open(path, 'rb') as f:
        var = pickle.load(f)
    return var

# class to index loaded dataset
class Dataset(data.Dataset):
    def __init__(self, path_to_pkl):
        self.data = load_from_pkl(path_to_pkl)
        self.ids = list(self.data.keys())

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        id = self.ids[index]
        dict = self.data[id]
        seq = dict["sequence"]
        pssm = dict["pssm"]
        length = len(seq)
        
        secstr = dict["secstr"]
        secstr_q3 = dict["secstr_q3"]

        return id,seq,pssm,length,secstr,secstr_q3
    
# Helper Function to pad a batch of protein sequences.
def pad_batch(DataLoaderBatch):
    """
    DataLoaderBatch should be a list of (sequence, target, length) tuples...
    Returns a padded tensor of sequences sorted from longest to shortest, 
    """
    batch_size = len(DataLoaderBatch)
    batch_split = list(zip(*DataLoaderBatch))

    ids, seqs, pssms, lengths, secstrs, secstrs_q3 = batch_split[0], batch_split[1], batch_split[2] ,batch_split[3], batch_split[4], batch_split[5]
    max_length = max(lengths)

    padded_seqs = np.ones((max_length, batch_size))*30
    padded_pssms = np.zeros((max_length,batch_size,22))
    padded_secstrs = np.ones((max_length,batch_size))*30
    padded_secstrs_q3 = np.ones((max_length,batch_size))*30
    
    for i, l in enumerate(lengths):
        padded_seqs[:l,i] = seqs[i][:l]
        padded_pssms[:l,i,:] = pssms[i][:l][:]
        padded_secstrs[:l,i] = secstrs[i][:l]
        padded_secstrs_q3[:l,i] = secstrs_q3[i][:l]
    return torch.tensor(padded_seqs).type(torch.LongTensor), torch.tensor(padded_pssms).type(torch.float), torch.tensor(lengths), torch.tensor(padded_secstrs).type(torch.LongTensor), torch.tensor(padded_secstrs_q3).type(torch.LongTensor)

### Load data

In [ ]:
# Batch Size for data parallelism
batch_size = 4

# Load train, test and validation data
train = Dataset("./Datasets/train.pkl")
val = Dataset("./Datasets/val.pkl")
test = Dataset("./Datasets/test.pkl")

# Data Generators for each set.
train_gen = data.DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=pad_batch,num_workers=1)
val_gen = data.DataLoader(val, batch_size=batch_size,shuffle=False,collate_fn=pad_batch,num_workers=1)
test_gen = data.DataLoader(test, batch_size=batch_size,shuffle=False,collate_fn=pad_batch,num_workers=1)

### Helper module for perceptron layer

In [ ]:
# Dense Layer
class Dense(nn.Module):
    def __init__(self,input_dim,output_dim,dropout = 0.2,normalization = True,activation = F.relu):
        super(Dense, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.dropout_rate = dropout
        self.activation = activation
        self.n_state = normalization        
        self.linear = nn.Linear(self.input_dim,self.output_dim)
        self.layernorm = nn.LayerNorm(self.output_dim)
        self.dropout = nn.Dropout(p=self.dropout_rate)
        
    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        if self.n_state:
            x = self.layernorm(x)
        x = self.dropout(x)
        return x

### Helper module for Bidirectional GRU 

In [ ]:
# Bidirectional GRU
class gru_bidirectional(nn.Module):
    def __init__(self,inp,out,dropout=0.1,activation=F.relu):
        super(gru_bidirectional,self).__init__()
        self.gru = nn.GRU(inp,out,bidirectional=True)
        self.dropout_rate = dropout
        self.layernorm = nn.LayerNorm(2*out)
        self.activation = activation
        self.input_dim = inp
        self.output_dim = out
        self.dropout = nn.Dropout(p=self.dropout_rate)

    def forward(self,x,lengths):
        x_ = pack_padded_sequence(x,lengths,enforce_sorted=False)
        out,_ = self.gru(x_)
        out,_ = pad_packed_sequence(out)
        out = self.activation(out)
        out = self.layernorm(out)
        if self.input_dim == 2*self.output_dim:
            out = out+x
        out = self.dropout(out)
        return out

### The Neural network definition

The structure of the neural network is defined here

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        '''
        Fill here
        '''
        
    def forward(self,x,pssm,x_lengths):
        '''
        Forward Pass here
        '''

### Initialize the network, Optimizer and the loss function

In [ ]:
network = Net().to(device)
criterion = nn.CrossEntropyLoss(ignore_index=30)
optimizer = torch.optim.Adam(network.parameters(),lr=0.001)

### Train Epoch helper Function

In [ ]:
def train_epoch(generator):
    #     Set the network in to train mode
    #     Enables dropout. 
    network.train()
    total_loss = 0.
    start_time = time.time() # Track training time.
    prediction = []
    g_t = []
    for seq,pssm,length,q8,q3 in (generator):
        '''
        Fill Training steps here
        
        variable loss should contain loss
        variable output should contain predictions generated by the network in the format [length,batch_size,8]
        '''
        total_loss += loss.cpu().detach().numpy()/batch_size   # Track loss for analysis
        
        # store all predictions to calculate accuracy at end of epoch
        for i in range(len(length)):
            prediction.append(output[0:length[i],i,:].cpu().detach().numpy())
            g_t.append(q8[0:length[i],i].cpu().detach().numpy())

    predictions = np.argmax(np.concatenate(prediction,axis=0),axis=-1)
    g_ts = np.concatenate(g_t)
    
    # calculate accuracy
    acc = accuracy_score(predictions,g_ts) 
    end_time = time.time()
    return end_time-start_time , total_loss, acc

### Evaluation Epoch Helper function

In [ ]:
def evaluate(generator, print_acc_report=False,output_sample=False):
    
    with torch.no_grad(): # disable gradients as no optimization done during evaluation
        network.eval()  # set network to evaluation mode to disable dropouts
        total_loss = 0.
        start_time = time.time() # Track evaluation time
        prediction = []
        g_t = []   
        for seq,pssm,length,q8,q3 in (generator):
            
            #    Get predictions
            output = network(seq.to(device),pssm.to(device),length.to(device))
            
            #    Calculate loss for analysis
            loss = criterion(output.view(-1,8).to(device),q8.view(-1).to(device))        
            total_loss += loss.cpu().detach().numpy()/batch_size
            for i in range(len(length)):
                prediction.append(output[0:length[i],i,:].cpu().detach().numpy())
                g_t.append(q8[0:length[i],i].cpu().detach().numpy())
                
        #   calculate  accuracy for analysis       
        predictions = np.argmax(np.concatenate(prediction,axis=0),axis=-1)
        g_ts = np.concatenate(g_t)
        acc = accuracy_score(predictions,g_ts) 
        end_time = time.time()
        if print_acc_report:
            print (classification_report(g_ts,predictions,target_names=target_names))
    return end_time-start_time, total_loss, acc

### Train!!!

In [ ]:
train_acc_history = []
val_acc_history = []
train_loss_history = []
val_loss_history = []
for i in range(40):
    #     Train epoch
    run_time,train_loss,train_acc = train_epoch(train_gen)
    
    #     Validation step
    _,val_loss,val_acc = evaluate(val_gen)
    
    #     Track performance of network during training
    train_acc_history.append(train_acc)
    val_acc_history.append(val_acc)
    train_loss_history.append(train_loss)
    val_loss_history.append(val_loss)
    
    #     Print performance metrics
    print ("Epoch : "+str(i)+", Train_loss : "+str(train_loss)+", Train_acc : "+str(train_acc)+", Val_acc : "+ str(val_acc))

### Evaluate on test set

In [ ]:
_,_,test_accuracy = evaluate(test_gen,True,True)
print ("Test Accuracy - ",test_accuracy)

### Visualize training

In [ ]:
plt.plot(train_acc_history,label = "Train_accuracy")
plt.plot(val_acc_history,label = "Validation_accuracy")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy in %")
plt.title("Performance with time")
plt.show()

In [ ]:
plt.plot(train_loss_history)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss with time")
plt.show()